## Modelos Supervisados: tuning de parámetros con validación cruzada, comparación de modelos

Vamos a utilizar datos obtenidos de una citología de cáncer de mama. Enfermedades como el cáncer de mama, además de ser muy graves, se definen por ser de difícil diagnosis. Cuanto antes se tiene un diagnóstico, antes se puede comenzar un tratamiento. 

El gran número de estudios que debe sufrir el paciente consumen tiempo y son costosos. En el caso de las citologías se miden variables como el radio, concavidad, simetría de las células.
Ciertas variables pueden tener mayor peso que otras, o una cierta combinación de variables puede ser determinante para saber el diagnóstico del paciente. Este es un ejemplo donde el aprendizaje computacional juega un papel importante.

Vamos a entrenar un modelo con el algoritmo Support Vector Machine con este conjunto de datos. Las variables son: 


Variable de clase: Diagnosis (M = malignant, B = benign)  

Variables calculadas para cada núcleo celular:

+ a) radius (mean of distances from center to points on the perimeter) 
+ b) texture (standard deviation of gray-scale values) 
+ c) perimeter 
+ d) area 
+ e) smoothness (local variation in radius lengths) 
+ f) compactness (perimeter^2 / area - 1.0) 
+ g) concavity (severity of concave portions of the contour) 
+ h) concave points (number of concave portions of the contour) 
+ i) symmetry 
+ j) fractal dimension ("coastline approximation" - 1)



In [1]:
import warnings
warnings.filterwarnings('ignore')

Vamos a leer los datos del archivo y a reemplazar los valores 2 y 4 de la Clase (benigno, maligno) por 0 y 1, ya que son valores más intuitivos y mejor comprendidos por la máquina. Nos quedaremos con las filas que tienen datos completos. Veamos cómo queda el dataset:

In [2]:
#Importamos librerías

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
import seaborn as sns

df = pd.read_csv(r'C:\Users\maria.pineiro.pereda\Desktop\formacion_fp\Practica_Supervisado\Data\breast-cancer-wisconsin.data', header=None, names=['ID', 'Clump Thickness', 'Uniformity of Cell Size', 'Uniformity of Cell Shape', 'Marginal Adhesion', 'Single Epithelial Cell Size', 'Bare Nuclei', 'Bland Chromatin', 'Normal Nucleoli', 'Mitoses', 'Class'])

df = df.set_index('ID')

df['Class'].replace(2, 0, inplace=True)

df['Class'].replace(4, 1, inplace=True)

df.replace('?', np.nan, inplace=True)

df.dropna(inplace=True)

print(df.head())

         Clump Thickness  Uniformity of Cell Size  Uniformity of Cell Shape  \
ID                                                                            
1000025                5                        1                         1   
1002945                5                        4                         4   
1015425                3                        1                         1   
1016277                6                        8                         8   
1017023                4                        1                         1   

         Marginal Adhesion  Single Epithelial Cell Size Bare Nuclei  \
ID                                                                    
1000025                  1                            2           1   
1002945                  5                            7          10   
1015425                  1                            2           2   
1016277                  1                            3           4   
1017023             

Vamos a necesitar dividir los datos en un grupo de entrenamiento y otro de validación. Usaremos la proporción 70-30.

In [3]:

#Dividimos entre training y test

y = df['Class']
df.drop('Class', axis=1, inplace=True)
X = df
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)


#### Probamos a entrenar un clasificador Support Vector Machine

Antes de entrenar los modelos, usaremos la función Pipeline, que unifica varios pasos (escalar o normalizar datos y definir el modelo) en uno solo.

In [4]:
# Vamos a probar el clasificador Support Vector Machine


pipeline_svc = Pipeline([('scl', StandardScaler()),
                     ('clf', SVC(random_state=1))])


#Entrenamos el pipeline con los datos

pipeline_svc.fit(X_train, y_train)

scores = cross_val_score(estimator=pipeline_svc,
                         X=X_train,
                         y=y_train,
                         cv=10,
                         n_jobs=1)

print('-->  Accuracy del entrenamiento: %.3f +/- %.3f' %(np.mean(scores), np.std(scores)))




C:\Users\maria.pineiro.pereda\.conda\envs\adeida64\lib\site-packages\sklearn\preprocessing\data.py:645: DataConversionWarning: Data with input dtype int64, object were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\maria.pineiro.pereda\.conda\envs\adeida64\lib\site-packages\sklearn\base.py:467: DataConversionWarning: Data with input dtype int64, object were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)
C:\Users\maria.pineiro.pereda\.conda\envs\adeida64\lib\site-packages\sklearn\preprocessing\data.py:645: DataConversionWarning: Data with input dtype int64, object were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\maria.pineiro.pereda\.conda\envs\adeida64\lib\site-packages\sklearn\base.py:467: DataConversionWarning: Data with input dtype int64, object were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)
C:\Users\maria

-->  Accuracy del entrenamiento: 0.975 +/- 0.021


C:\Users\maria.pineiro.pereda\.conda\envs\adeida64\lib\site-packages\sklearn\preprocessing\data.py:645: DataConversionWarning: Data with input dtype int64, object were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\maria.pineiro.pereda\.conda\envs\adeida64\lib\site-packages\sklearn\base.py:467: DataConversionWarning: Data with input dtype int64, object were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)
C:\Users\maria.pineiro.pereda\.conda\envs\adeida64\lib\site-packages\sklearn\pipeline.py:511: DataConversionWarning: Data with input dtype int64, object were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)


Para elegir los parámetros óptimos, generamos un *grid* de parámetros con un rango de valores y usamos GridSearchCV para buscar en ese rango el conjunto de parámetros óptimo. En el grid tenemos dos posibilidades: encontrar un modelo lineal con un valor de C que se obtendrá por tuning, o encontrar un modelo no lineal con dos parámetros que deben ser elegidos por tuning: gamma y C. Elegimos métrica de accuracy, 10 para validación cruzada.

In [5]:

#Tuning de parámetros

param_range = [0.0001, 0.001, 0.01, 0.1, 1.0, 10.0, 100.0, 1000.0]

param_grid = [{'clf__C': param_range,
               'clf__kernel': ['linear']},
              {'clf__C': param_range,
               'clf__gamma': param_range,
               'clf__kernel': ['rbf']}]

gs = GridSearchCV(estimator=pipeline_svc,
                  param_grid=param_grid,
                  scoring='accuracy',
                  cv=10,
                  n_jobs=1)

Veamos cuál es el mejor score que obtenemos y con qué parámetros:

In [6]:
## 
gs = gs.fit(X_train, y_train)

print('--> Mejor score obtenido: ',gs.best_score_)
print('--> Mejores parámetros con los que obtenemos ese score: \n',gs.best_params_)


C:\Users\maria.pineiro.pereda\.conda\envs\adeida64\lib\site-packages\sklearn\preprocessing\data.py:645: DataConversionWarning: Data with input dtype int64, object were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\maria.pineiro.pereda\.conda\envs\adeida64\lib\site-packages\sklearn\base.py:467: DataConversionWarning: Data with input dtype int64, object were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)
C:\Users\maria.pineiro.pereda\.conda\envs\adeida64\lib\site-packages\sklearn\pipeline.py:331: DataConversionWarning: Data with input dtype int64, object were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
C:\Users\maria.pineiro.pereda\.conda\envs\adeida64\lib\site-packages\sklearn\pipeline.py:331: DataConversionWarning: Data with input dtype int64, object were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
C:\Users\maria.pineiro.pereda\.conda\envs

C:\Users\maria.pineiro.pereda\.conda\envs\adeida64\lib\site-packages\sklearn\pipeline.py:331: DataConversionWarning: Data with input dtype int64, object were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
C:\Users\maria.pineiro.pereda\.conda\envs\adeida64\lib\site-packages\sklearn\preprocessing\data.py:645: DataConversionWarning: Data with input dtype int64, object were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\maria.pineiro.pereda\.conda\envs\adeida64\lib\site-packages\sklearn\base.py:467: DataConversionWarning: Data with input dtype int64, object were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)
C:\Users\maria.pineiro.pereda\.conda\envs\adeida64\lib\site-packages\sklearn\pipeline.py:331: DataConversionWarning: Data with input dtype int64, object were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
C:\Users\maria.pineiro.pereda\.conda\envs

C:\Users\maria.pineiro.pereda\.conda\envs\adeida64\lib\site-packages\sklearn\pipeline.py:331: DataConversionWarning: Data with input dtype int64, object were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
C:\Users\maria.pineiro.pereda\.conda\envs\adeida64\lib\site-packages\sklearn\preprocessing\data.py:645: DataConversionWarning: Data with input dtype int64, object were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\maria.pineiro.pereda\.conda\envs\adeida64\lib\site-packages\sklearn\base.py:467: DataConversionWarning: Data with input dtype int64, object were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)
C:\Users\maria.pineiro.pereda\.conda\envs\adeida64\lib\site-packages\sklearn\pipeline.py:331: DataConversionWarning: Data with input dtype int64, object were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
C:\Users\maria.pineiro.pereda\.conda\envs

C:\Users\maria.pineiro.pereda\.conda\envs\adeida64\lib\site-packages\sklearn\preprocessing\data.py:645: DataConversionWarning: Data with input dtype int64, object were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\maria.pineiro.pereda\.conda\envs\adeida64\lib\site-packages\sklearn\base.py:467: DataConversionWarning: Data with input dtype int64, object were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)
C:\Users\maria.pineiro.pereda\.conda\envs\adeida64\lib\site-packages\sklearn\pipeline.py:331: DataConversionWarning: Data with input dtype int64, object were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
C:\Users\maria.pineiro.pereda\.conda\envs\adeida64\lib\site-packages\sklearn\pipeline.py:331: DataConversionWarning: Data with input dtype int64, object were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
C:\Users\maria.pineiro.pereda\.conda\envs

C:\Users\maria.pineiro.pereda\.conda\envs\adeida64\lib\site-packages\sklearn\base.py:467: DataConversionWarning: Data with input dtype int64, object were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)
C:\Users\maria.pineiro.pereda\.conda\envs\adeida64\lib\site-packages\sklearn\pipeline.py:331: DataConversionWarning: Data with input dtype int64, object were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
C:\Users\maria.pineiro.pereda\.conda\envs\adeida64\lib\site-packages\sklearn\pipeline.py:331: DataConversionWarning: Data with input dtype int64, object were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
C:\Users\maria.pineiro.pereda\.conda\envs\adeida64\lib\site-packages\sklearn\preprocessing\data.py:645: DataConversionWarning: Data with input dtype int64, object were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\maria.pineiro.pereda\.conda\envs

...

C:\Users\maria.pineiro.pereda\.conda\envs\adeida64\lib\site-packages\sklearn\pipeline.py:331: DataConversionWarning: Data with input dtype int64, object were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
C:\Users\maria.pineiro.pereda\.conda\envs\adeida64\lib\site-packages\sklearn\preprocessing\data.py:645: DataConversionWarning: Data with input dtype int64, object were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\maria.pineiro.pereda\.conda\envs\adeida64\lib\site-packages\sklearn\base.py:467: DataConversionWarning: Data with input dtype int64, object were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)
C:\Users\maria.pineiro.pereda\.conda\envs\adeida64\lib\site-packages\sklearn\pipeline.py:331: DataConversionWarning: Data with input dtype int64, object were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
C:\Users\maria.pineiro.pereda\.conda\envs

C:\Users\maria.pineiro.pereda\.conda\envs\adeida64\lib\site-packages\sklearn\preprocessing\data.py:645: DataConversionWarning: Data with input dtype int64, object were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\maria.pineiro.pereda\.conda\envs\adeida64\lib\site-packages\sklearn\base.py:467: DataConversionWarning: Data with input dtype int64, object were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)
C:\Users\maria.pineiro.pereda\.conda\envs\adeida64\lib\site-packages\sklearn\pipeline.py:331: DataConversionWarning: Data with input dtype int64, object were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
C:\Users\maria.pineiro.pereda\.conda\envs\adeida64\lib\site-packages\sklearn\pipeline.py:331: DataConversionWarning: Data with input dtype int64, object were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
C:\Users\maria.pineiro.pereda\.conda\envs

C:\Users\maria.pineiro.pereda\.conda\envs\adeida64\lib\site-packages\sklearn\pipeline.py:331: DataConversionWarning: Data with input dtype int64, object were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
C:\Users\maria.pineiro.pereda\.conda\envs\adeida64\lib\site-packages\sklearn\preprocessing\data.py:645: DataConversionWarning: Data with input dtype int64, object were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\maria.pineiro.pereda\.conda\envs\adeida64\lib\site-packages\sklearn\base.py:467: DataConversionWarning: Data with input dtype int64, object were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)
C:\Users\maria.pineiro.pereda\.conda\envs\adeida64\lib\site-packages\sklearn\pipeline.py:331: DataConversionWarning: Data with input dtype int64, object were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
C:\Users\maria.pineiro.pereda\.conda\envs

C:\Users\maria.pineiro.pereda\.conda\envs\adeida64\lib\site-packages\sklearn\preprocessing\data.py:645: DataConversionWarning: Data with input dtype int64, object were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\maria.pineiro.pereda\.conda\envs\adeida64\lib\site-packages\sklearn\base.py:467: DataConversionWarning: Data with input dtype int64, object were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)
C:\Users\maria.pineiro.pereda\.conda\envs\adeida64\lib\site-packages\sklearn\pipeline.py:331: DataConversionWarning: Data with input dtype int64, object were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
C:\Users\maria.pineiro.pereda\.conda\envs\adeida64\lib\site-packages\sklearn\pipeline.py:331: DataConversionWarning: Data with input dtype int64, object were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
C:\Users\maria.pineiro.pereda\.conda\envs

--> Mejor score obtenido:  0.9769874476987448
--> Mejores parámetros con los que obtenemos ese score: 
 {'clf__C': 0.01, 'clf__kernel': 'linear'}


Vemos que con un modelo lineal y un C de 0.01 obtenemos el modelo que mejor predice. Vamos a usar esos parámetros para entrenar el modelo entonces, y a probar ese modelo entrenado de forma óptima en los datos de validación que separamos al principio.

In [7]:

#Usamos esos mejores parámetros

clf_svc = gs.best_estimator_

#Valores finales

clf_svc.fit(X_train, y_train)

scores = cross_val_score(estimator=clf_svc,
                         X=X_train,
                         y=y_train,
                         cv=10,
                         n_jobs=1)

print('-->  Accuracy final de entrenamiento: %.3f +/- %.3f' %(np.mean(scores), np.std(scores)))


print('--> Accuracy final de test: %.5f' % clf_svc.score(X_test,y_test))




C:\Users\maria.pineiro.pereda\.conda\envs\adeida64\lib\site-packages\sklearn\preprocessing\data.py:645: DataConversionWarning: Data with input dtype int64, object were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\maria.pineiro.pereda\.conda\envs\adeida64\lib\site-packages\sklearn\base.py:467: DataConversionWarning: Data with input dtype int64, object were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)
C:\Users\maria.pineiro.pereda\.conda\envs\adeida64\lib\site-packages\sklearn\preprocessing\data.py:645: DataConversionWarning: Data with input dtype int64, object were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\maria.pineiro.pereda\.conda\envs\adeida64\lib\site-packages\sklearn\base.py:467: DataConversionWarning: Data with input dtype int64, object were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)
C:\Users\maria

-->  Accuracy final de entrenamiento: 0.977 +/- 0.020
--> Accuracy final de test: 0.95610


C:\Users\maria.pineiro.pereda\.conda\envs\adeida64\lib\site-packages\sklearn\pipeline.py:511: DataConversionWarning: Data with input dtype int64, object were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
C:\Users\maria.pineiro.pereda\.conda\envs\adeida64\lib\site-packages\sklearn\preprocessing\data.py:645: DataConversionWarning: Data with input dtype int64, object were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\maria.pineiro.pereda\.conda\envs\adeida64\lib\site-packages\sklearn\base.py:467: DataConversionWarning: Data with input dtype int64, object were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)
C:\Users\maria.pineiro.pereda\.conda\envs\adeida64\lib\site-packages\sklearn\pipeline.py:511: DataConversionWarning: Data with input dtype int64, object were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
C:\Users\maria.pineiro.pereda\.conda\envs

El resultado es 0.956 en generalización (en datos de test). Es lógico que el accuracy sea más bajo que en training, ya que estos datos de test no han sido vistos previamente por el modelo. Aun así, se trata de un accuracy muy bueno.

### OTROS MODELOS QUE PODEMOS ENTRENAR ... 

#### Vamos a entrenar un modelo de Regresión Logística de la misma forma que entrenamos el SVM

Prueba a entrenar un modelo de regresión logística

In [8]:

from sklearn.linear_model import LogisticRegression

#Construimos pipeline

pipeline_lr = Pipeline([('scl', StandardScaler()),
                    ('clf', LogisticRegression(penalty='l2', tol=0.0001, C=1.0, random_state=1, max_iter=1000, n_jobs=-1))]) ## Logistic Regression, buscar parámetros que usa por defecto


#Entrenamos con los datos

pipeline_lr.fit( X_train ,y_train  )#completar

scores = cross_val_score(estimator=pipeline_lr,
                         X= X_train, #completar
                         y= y_train , #completar
                         cv=10 , #completar
                         n_jobs=1)

print('--> Accuracy entrenamiento: %.3f +/- %.3f' %(np.mean( scores ), np.std( scores )))#completar



C:\Users\maria.pineiro.pereda\.conda\envs\adeida64\lib\site-packages\sklearn\preprocessing\data.py:645: DataConversionWarning: Data with input dtype int64, object were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\maria.pineiro.pereda\.conda\envs\adeida64\lib\site-packages\sklearn\base.py:467: DataConversionWarning: Data with input dtype int64, object were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)
C:\Users\maria.pineiro.pereda\.conda\envs\adeida64\lib\site-packages\sklearn\linear_model\logistic.py:1300: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 4.
  " = {}.".format(effective_n_jobs(self.n_jobs)))
C:\Users\maria.pineiro.pereda\.conda\envs\adeida64\lib\site-packages\sklearn\preprocessing\data.py:645: DataConversionWarning: Data with input dtype int64, object were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)

--> Accuracy entrenamiento: 0.971 +/- 0.017


C:\Users\maria.pineiro.pereda\.conda\envs\adeida64\lib\site-packages\sklearn\base.py:467: DataConversionWarning: Data with input dtype int64, object were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)
C:\Users\maria.pineiro.pereda\.conda\envs\adeida64\lib\site-packages\sklearn\linear_model\logistic.py:1300: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 4.
  " = {}.".format(effective_n_jobs(self.n_jobs)))
C:\Users\maria.pineiro.pereda\.conda\envs\adeida64\lib\site-packages\sklearn\pipeline.py:511: DataConversionWarning: Data with input dtype int64, object were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)


In [ ]:

#Tunning de hiperparámetros. Probamos con l1 y l2 para regularización, y diferentes rangos de C y tol

param_range = [0.0001, 0.001, 0.01, 0.1, 1.0, 10.0, 100.0, 1000.0]
param_range_small = [0.000001, 0.00001, 0.0001, 0.001, 0.01, 0.1, 1.0, 10.0]

param_grid_lr = [{'clf__penalty': ['l1'],
               'clf__C': param_range,
               'clf__tol': param_range_small},
              {'clf__penalty': ['l2'],
               'clf__C': param_range,
               'clf__tol': param_range_small}]

gs_lr = GridSearchCV(estimator=pipeline_lr , #completar
                  param_grid= param_grid_lr, #completar
                  scoring='accuracy',
                  cv= 10 , #completar
                  n_jobs=1)

gs_lr = gs_lr.fit( X_train ,  y_train ) #completar

print('--> Mejor score de los parámetros de tunning: ',   gs_lr.best_score_) #completar

print('--> Mejores parámetros: \n',    gs_lr.best_params_)#completar


C:\Users\maria.pineiro.pereda\.conda\envs\adeida64\lib\site-packages\sklearn\preprocessing\data.py:645: DataConversionWarning: Data with input dtype int64, object were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\maria.pineiro.pereda\.conda\envs\adeida64\lib\site-packages\sklearn\base.py:467: DataConversionWarning: Data with input dtype int64, object were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)
C:\Users\maria.pineiro.pereda\.conda\envs\adeida64\lib\site-packages\sklearn\linear_model\logistic.py:1300: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 4.
  " = {}.".format(effective_n_jobs(self.n_jobs)))
C:\Users\maria.pineiro.pereda\.conda\envs\adeida64\lib\site-packages\sklearn\pipeline.py:331: DataConversionWarning: Data with input dtype int64, object were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
C:\Users\m

C:\Users\maria.pineiro.pereda\.conda\envs\adeida64\lib\site-packages\sklearn\pipeline.py:331: DataConversionWarning: Data with input dtype int64, object were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
C:\Users\maria.pineiro.pereda\.conda\envs\adeida64\lib\site-packages\sklearn\preprocessing\data.py:645: DataConversionWarning: Data with input dtype int64, object were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\maria.pineiro.pereda\.conda\envs\adeida64\lib\site-packages\sklearn\base.py:467: DataConversionWarning: Data with input dtype int64, object were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)
C:\Users\maria.pineiro.pereda\.conda\envs\adeida64\lib\site-packages\sklearn\linear_model\logistic.py:1300: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 4.
  " = {}.".format(effective_n_jobs(self.n_jobs)))
C:\Users\m

C:\Users\maria.pineiro.pereda\.conda\envs\adeida64\lib\site-packages\sklearn\base.py:467: DataConversionWarning: Data with input dtype int64, object were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)
C:\Users\maria.pineiro.pereda\.conda\envs\adeida64\lib\site-packages\sklearn\linear_model\logistic.py:1300: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 4.
  " = {}.".format(effective_n_jobs(self.n_jobs)))
C:\Users\maria.pineiro.pereda\.conda\envs\adeida64\lib\site-packages\sklearn\pipeline.py:331: DataConversionWarning: Data with input dtype int64, object were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
C:\Users\maria.pineiro.pereda\.conda\envs\adeida64\lib\site-packages\sklearn\pipeline.py:331: DataConversionWarning: Data with input dtype int64, object were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
C:\Users\maria.pineir

C:\Users\maria.pineiro.pereda\.conda\envs\adeida64\lib\site-packages\sklearn\base.py:467: DataConversionWarning: Data with input dtype int64, object were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)
C:\Users\maria.pineiro.pereda\.conda\envs\adeida64\lib\site-packages\sklearn\linear_model\logistic.py:1300: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 4.
  " = {}.".format(effective_n_jobs(self.n_jobs)))
C:\Users\maria.pineiro.pereda\.conda\envs\adeida64\lib\site-packages\sklearn\pipeline.py:331: DataConversionWarning: Data with input dtype int64, object were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
C:\Users\maria.pineiro.pereda\.conda\envs\adeida64\lib\site-packages\sklearn\pipeline.py:331: DataConversionWarning: Data with input dtype int64, object were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
C:\Users\maria.pineir

C:\Users\maria.pineiro.pereda\.conda\envs\adeida64\lib\site-packages\sklearn\pipeline.py:331: DataConversionWarning: Data with input dtype int64, object were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
C:\Users\maria.pineiro.pereda\.conda\envs\adeida64\lib\site-packages\sklearn\preprocessing\data.py:645: DataConversionWarning: Data with input dtype int64, object were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\maria.pineiro.pereda\.conda\envs\adeida64\lib\site-packages\sklearn\base.py:467: DataConversionWarning: Data with input dtype int64, object were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)
C:\Users\maria.pineiro.pereda\.conda\envs\adeida64\lib\site-packages\sklearn\linear_model\logistic.py:1300: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 4.
  " = {}.".format(effective_n_jobs(self.n_jobs)))
C:\Users\m

...

C:\Users\maria.pineiro.pereda\.conda\envs\adeida64\lib\site-packages\sklearn\base.py:467: DataConversionWarning: Data with input dtype int64, object were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)
C:\Users\maria.pineiro.pereda\.conda\envs\adeida64\lib\site-packages\sklearn\linear_model\logistic.py:1300: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 4.
  " = {}.".format(effective_n_jobs(self.n_jobs)))
C:\Users\maria.pineiro.pereda\.conda\envs\adeida64\lib\site-packages\sklearn\pipeline.py:331: DataConversionWarning: Data with input dtype int64, object were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
C:\Users\maria.pineiro.pereda\.conda\envs\adeida64\lib\site-packages\sklearn\pipeline.py:331: DataConversionWarning: Data with input dtype int64, object were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
C:\Users\maria.pineir

C:\Users\maria.pineiro.pereda\.conda\envs\adeida64\lib\site-packages\sklearn\base.py:467: DataConversionWarning: Data with input dtype int64, object were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)
C:\Users\maria.pineiro.pereda\.conda\envs\adeida64\lib\site-packages\sklearn\linear_model\logistic.py:1300: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 4.
  " = {}.".format(effective_n_jobs(self.n_jobs)))
C:\Users\maria.pineiro.pereda\.conda\envs\adeida64\lib\site-packages\sklearn\pipeline.py:331: DataConversionWarning: Data with input dtype int64, object were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
C:\Users\maria.pineiro.pereda\.conda\envs\adeida64\lib\site-packages\sklearn\pipeline.py:331: DataConversionWarning: Data with input dtype int64, object were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
C:\Users\maria.pineir

C:\Users\maria.pineiro.pereda\.conda\envs\adeida64\lib\site-packages\sklearn\preprocessing\data.py:645: DataConversionWarning: Data with input dtype int64, object were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\maria.pineiro.pereda\.conda\envs\adeida64\lib\site-packages\sklearn\base.py:467: DataConversionWarning: Data with input dtype int64, object were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)
C:\Users\maria.pineiro.pereda\.conda\envs\adeida64\lib\site-packages\sklearn\linear_model\logistic.py:1300: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 4.
  " = {}.".format(effective_n_jobs(self.n_jobs)))
C:\Users\maria.pineiro.pereda\.conda\envs\adeida64\lib\site-packages\sklearn\pipeline.py:331: DataConversionWarning: Data with input dtype int64, object were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
C:\Users\m

C:\Users\maria.pineiro.pereda\.conda\envs\adeida64\lib\site-packages\sklearn\pipeline.py:331: DataConversionWarning: Data with input dtype int64, object were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
C:\Users\maria.pineiro.pereda\.conda\envs\adeida64\lib\site-packages\sklearn\pipeline.py:331: DataConversionWarning: Data with input dtype int64, object were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
C:\Users\maria.pineiro.pereda\.conda\envs\adeida64\lib\site-packages\sklearn\preprocessing\data.py:645: DataConversionWarning: Data with input dtype int64, object were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\maria.pineiro.pereda\.conda\envs\adeida64\lib\site-packages\sklearn\base.py:467: DataConversionWarning: Data with input dtype int64, object were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)
C:\Users\maria.pineiro.pereda\.conda\envs

C:\Users\maria.pineiro.pereda\.conda\envs\adeida64\lib\site-packages\sklearn\base.py:467: DataConversionWarning: Data with input dtype int64, object were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)
C:\Users\maria.pineiro.pereda\.conda\envs\adeida64\lib\site-packages\sklearn\linear_model\logistic.py:1300: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 4.
  " = {}.".format(effective_n_jobs(self.n_jobs)))
C:\Users\maria.pineiro.pereda\.conda\envs\adeida64\lib\site-packages\sklearn\pipeline.py:331: DataConversionWarning: Data with input dtype int64, object were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
C:\Users\maria.pineiro.pereda\.conda\envs\adeida64\lib\site-packages\sklearn\pipeline.py:331: DataConversionWarning: Data with input dtype int64, object were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
C:\Users\maria.pineir


Vamos a usar los mejores parámetros para entrenar el modelo final y probarlo en el conjunto de datos de validación.

In [ ]:

#Usamos los mejores parámetros

clf_lr =  gs_lr.best_estimator_ #completar


#Scores finales
clf_lr.fit(X_train , y_train )  #completar
scores_lr = cross_val_score(estimator=clf_lr ,#completar
                         X= X_train ,#completar
                         y= y_train,#completar
                         cv= 10,#completar
                         n_jobs=1)

print('--> Accuracy final de entrenamiento : %.3f +/- %.3f' %(np.mean(scores_lr ), np.std( scores_lr))) #completar

print('--> Accuracy de test x: %.5f' % clf_lr.score( X_test , y_test  )) #completar

## Resumen 

In [ ]:
## Con regresiónlogística
from sklearn.linear_model import LogisticRegression

#Construimos pipeline

pipeline_lr = Pipeline([('scl', StandardScaler()),
                    ('clf', LogisticRegression(penalty='l2', tol=0.0001, C=1.0, random_state=1, max_iter=1000, n_jobs=-1))]) ## Logistic Regression, buscar parámetros que usa por defecto


#Entrenamos con los datos

pipeline_lr.fit( X_train ,y_train  )#completar

#Vemos como de bueno es el modelo con los datos
scores = cross_val_score(estimator=pipeline_lr,
                         X= X_train, #completar
                         y= y_train , #completar
                         cv=10 , #completar
                         n_jobs=1)

print('--> Accuracy entrenamiento: %.3f +/- %.3f' %(np.mean( scores ), np.std( scores )))#completar

##Damos nuevos datos al modelo que no ha visto nunca
print('--> Accuracy de test x: %.5f' % pipeline_lr.score( X_test , y_test  )) #completar

In [ ]:
## Con SVM
# Vamos a probar el clasificador Support Vector Machine


pipeline_svc = Pipeline([('scl', StandardScaler()),
                     ('clf', SVC(random_state=1))])


#Entrenamos el pipeline con los datos

pipeline_svc.fit(X_train, y_train)

scores = cross_val_score(estimator=pipeline_svc,
                         X=X_train,
                         y=y_train,
                         cv=10,
                         n_jobs=1)

print('-->  Accuracy del entrenamiento: %.3f +/- %.3f' %(np.mean(scores), np.std(scores)))


##Damos nuevos datos al modelo que no ha visto nunca
print('--> Accuracy de test x: %.5f' % pipeline_svc.score( X_test , y_test  )) #completar

## ¿Cómo usamos el modelo que hemos entrenado?

In [ ]:
#Guardamos el modelo regr logistica
import pickle
filename = 'clf_brca_model.sav'
pickle.dump(pipeline_lr, open(filename, 'wb'))

In [ ]:
#Guradamos el modelo SVM
filename = 'svc_brca_model.sav'
pickle.dump(pipeline_svc, open(filename, 'wb'))